In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.layers import ConvLSTM2D, TimeDistributed, Dense, Flatten
from tensorflow.keras.models import Sequential
import cv2
import numpy as np


In [ ]:
# Load Xception model 
xception_model = Xception(weights='imagenet', include_top=False, pooling='avg', input_shape=(299, 299, 3))


In [ ]:
# ConvLSTM model
model = Sequential()
model.add(TimeDistributed(xception_model, input_shape=(None, 299, 299, 3)))
model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3), padding='same', return_sequences=False))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
def extract_frames(video_path, num_frames=20):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = max(1, total_frames // num_frames)
    
    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_interval)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (299, 299))
            frame = preprocess_input(frame)
            frames.append(frame)
        else:
            break
    cap.release()
    return np.array(frames)


In [ ]:
video_path = 'video.mp4'
frames = extract_frames(video_path)
frames = np.expand_dims(frames, axis=0)  # Add batch dimension

# Predict deepfake probability
prediction = model.predict(frames)
probability = prediction[0][0]
print(f'Deepfake Probability: {probability:.2f}')
